In [43]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import difflib
from pulp import *
import openpyxl
import glob

from datetime import *
today = date.today()
today = today.strftime("%m.%d.20%y")
today

# import datetime
# today = date.today()
# today = today - datetime.timedelta(days=1)
# today = today.strftime("%m.%d.20%y")
# today

'03.08.2022'

In [44]:
## FanDuel Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

fd = glob.glob('FanDuel-NBA*.csv')[0]
os.rename(fd, "FDSalaries_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FDSalaries_"+ contest + str(today)+".csv",
           contest_data_loc + "/"+ "FDSalaries_"+ contest + str(today)+".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("FDSalaries_"+ contest + str(today)+".csv")


In [67]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [68]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id']], 
              left_on='Player', 
              right_on='Nickname')

In [69]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id
0,Cameron Payne,PHX,ORL,33.0,PG/SG,"6,600",40.1,8.7,49.0,67.8,MAIN,Cameron Payne,PG/SG,72603-48567
1,CJ McCollum,NOP,MEM,37.0,SG,"8,600",48.0,8.6,55.9,6.8,MAIN,CJ McCollum,SG,72603-19067
2,Jonas Valanciunas,NOP,MEM,31.0,C,"7,000",40.9,7.9,48.7,9.8,MAIN,Jonas Valanciunas,C,72603-14503
3,LaMelo Ball,CHA,BKN,32.0,PG,"7,800",43.3,7.1,53.4,31.6,MAIN,LaMelo Ball,PG,72603-145485
4,Herbert Jones,NOP,MEM,34.0,PF,"4,600",29.2,5.8,38.5,11.2,MAIN,Herbert Jones,PF,72603-157848


In [70]:
# Add 'G' Column
# Add 'F' Column
df['G Position'] = np.where(df['FD Position'].str.contains("G"), 'G', "")
df['F Position'] = np.where(df['FD Position'].str.contains("F"), 'F', "")

df['G Position'] = df['G Position'].replace('', np.nan, regex=True)
df['F Position'] = df['F Position'].replace('', np.nan, regex=True)

df['list'] = df[['FD Position', 'G Position', 'F Position']].apply(lambda x: '/'.join(x.dropna()), axis=1)

# ## Update Salary to INT 
df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df['Name + ID'] = df['Id'] + ":" + df['Player']

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list,Name + ID
0,Cameron Payne,PHX,ORL,33.0,PG/SG,6600.0,40.1,8.7,49.0,67.8,MAIN,Cameron Payne,PG/SG,72603-48567,G,NaN,PG/SG/G,72603-48567:Cameron Payne
1,CJ McCollum,NOP,MEM,37.0,SG,8600.0,48.0,8.6,55.9,6.8,MAIN,CJ McCollum,SG,72603-19067,G,NaN,SG/G,72603-19067:CJ McCollum
2,Jonas Valanciunas,NOP,MEM,31.0,C,7000.0,40.9,7.9,48.7,9.8,MAIN,Jonas Valanciunas,C,72603-14503,NaN,NaN,C,72603-14503:Jonas Valanciunas
3,LaMelo Ball,CHA,BKN,32.0,PG,7800.0,43.3,7.1,53.4,31.6,MAIN,LaMelo Ball,PG,72603-145485,G,NaN,PG/G,72603-145485:LaMelo Ball
4,Herbert Jones,NOP,MEM,34.0,PF,4600.0,29.2,5.8,38.5,11.2,MAIN,Herbert Jones,PF,72603-157848,NaN,F,PF/F,72603-157848:Herbert Jones


In [71]:
df.to_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index=False)

df = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\FD_final_PlayerPool_ETR_'+ str(today) + ".csv")

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list,Name + ID
0,Cameron Payne,PHX,ORL,33.0,PG/SG,6600.0,40.1,8.7,49.0,67.8,MAIN,Cameron Payne,PG/SG,72603-48567,G,NaN,PG/SG/G,72603-48567:Cameron Payne
1,CJ McCollum,NOP,MEM,37.0,SG,8600.0,48.0,8.6,55.9,6.8,MAIN,CJ McCollum,SG,72603-19067,G,NaN,SG/G,72603-19067:CJ McCollum
2,Jonas Valanciunas,NOP,MEM,31.0,C,7000.0,40.9,7.9,48.7,9.8,MAIN,Jonas Valanciunas,C,72603-14503,NaN,NaN,C,72603-14503:Jonas Valanciunas
3,LaMelo Ball,CHA,BKN,32.0,PG,7800.0,43.3,7.1,53.4,31.6,MAIN,LaMelo Ball,PG,72603-145485,G,NaN,PG/G,72603-145485:LaMelo Ball
4,Herbert Jones,NOP,MEM,34.0,PF,4600.0,29.2,5.8,38.5,11.2,MAIN,Herbert Jones,PF,72603-157848,NaN,F,PF/F,72603-157848:Herbert Jones


# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG = 2
- TotalPosition_SG = 2
- TotalPosition_SF = 2
- TotalPosition_PF =2 
- TotalPosition_C =1


In [72]:
## Unnests FD Positions into each Row (explode in Python) ##


## Update Salary to INT 
#df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['FD Position'] = df['FD Position'].astype(str)
df['FD Position'] = df['FD Position'].str.split('/')

df = df.explode('FD Position')

df['Position + Id'] = df['FD Position'] + "_" + df['Id']

df.head()

df.to_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

In [73]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list,Name + ID,Position + Id
0,Cameron Payne,PHX,ORL,33.0,PG,6600.0,40.1,8.7,49.0,67.8,MAIN,Cameron Payne,PG/SG,72603-48567,G,NaN,PG/SG/G,72603-48567:Cameron Payne,PG_72603-48567
0,Cameron Payne,PHX,ORL,33.0,SG,6600.0,40.1,8.7,49.0,67.8,MAIN,Cameron Payne,PG/SG,72603-48567,G,NaN,PG/SG/G,72603-48567:Cameron Payne,SG_72603-48567
1,CJ McCollum,NOP,MEM,37.0,SG,8600.0,48.0,8.6,55.9,6.8,MAIN,CJ McCollum,SG,72603-19067,G,NaN,SG/G,72603-19067:CJ McCollum,SG_72603-19067
2,Jonas Valanciunas,NOP,MEM,31.0,C,7000.0,40.9,7.9,48.7,9.8,MAIN,Jonas Valanciunas,C,72603-14503,NaN,NaN,C,72603-14503:Jonas Valanciunas,C_72603-14503
3,LaMelo Ball,CHA,BKN,32.0,PG,7800.0,43.3,7.1,53.4,31.6,MAIN,LaMelo Ball,PG,72603-145485,G,NaN,PG/G,72603-145485:LaMelo Ball,PG_72603-145485


In [74]:
df = pd.read_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index_col=['Id', 'FD Position'], skipinitialspace=True)

legal_assignments = df.index # tuples of (name, pos)
name_set = df.index.unique(0) # a convenience

costs = df['FD Salary'].to_dict()
values = df['FD Points'].to_dict()

# set up LP
draft = pulp.LpVariable.dicts('selected', legal_assignments, cat='Binary')

prob = pulp.LpProblem('the draft', LpMaximize)

#obj
prob += pulp.lpSum([draft[n,p]*values[n,p] for (n,p) in legal_assignments])

#salary cap
prob += pulp.lpSum([draft[n,p]*costs[n,p] for (n, p) in legal_assignments]) <= 60000

# positions
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PG']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SG']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SF']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PF']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'C']) == 1

# use each player at most only once
result = []
for name in name_set:
    prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if n == name]) <= 1

prob.solve()

for idx in draft:
    if draft[idx].varValue > 0:
        print(idx[0], idx[1])
        
        result.append([idx[0], idx[1]])

C:\Users\vchang\Anaconda3\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


72603-48567 PG
72603-19067 SG
72603-14503 C
72603-145485 PG
72603-157848 PF
72603-84678 SF
72603-110340 PF
72603-9646 SF
72603-145541 SG


In [75]:
# new_list = result
# df = pd.DataFrame(new_list)
# test = df[[1,0]]
# test1 = test.transpose()
# test2 = test1.rename(columns=test1.iloc[0])
# test3 = test2.drop([1])
# df2 = test3[['PG', 'SG','SF','PF', 'C']]
# df2.to_csv("fd_result_" + str(today) + ".csv", index=None)

In [76]:
new_list = result
df = pd.DataFrame(new_list)
test = df[[1,0]]

# Create a lookup variable for [Position + Id]
test['lookup'] = test[1] + "_" + test[0]


# Merge PuLP output on [Position + Id] to pull in Player Name
df = pd.read_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv")
upload = pd.merge(test,
                 df[['Player', 'Position + Id', 'Name + ID']],
                 left_on = 'lookup',
                 right_on = 'Position + Id')


# Drop all columns not used in upload
upload2 = upload.drop([0, 'lookup', 'Player', 'Position + Id'], axis = 1)


# Transpose data output from vertical to horizontal
upload3 = upload2.transpose()


# Rename columns to first row (position values)
upload4 = upload3.rename(columns=upload3.iloc[0])


# Drop duplicate first row
upload5 = upload4.drop([1])


# Re-order the columns to the positions
upload6 = upload5[['PG', 'SG','SF','PF', 'C']]


# Save as a csv
upload6.to_csv("fd_result_" + str(today) + ".csv", index=None)

upload6

C:\Users\vchang\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PG,PG,SG,SG,SF,SF,PF,PF,C
Name + ID,72603-48567:Cameron Payne,72603-145485:LaMelo Ball,72603-19067:CJ McCollum,72603-145541:R.J. Hampton,72603-84678:Miles Bridges,72603-9646:Kevin Durant,72603-157848:Herbert Jones,72603-110340:Jaxson Hayes,72603-14503:Jonas Valanciunas


In [77]:
upload6

,PG,PG,SG,SG,SF,SF,PF,PF,C
Name + ID,72603-48567:Cameron Payne,72603-145485:LaMelo Ball,72603-19067:CJ McCollum,72603-145541:R.J. Hampton,72603-84678:Miles Bridges,72603-9646:Kevin Durant,72603-157848:Herbert Jones,72603-110340:Jaxson Hayes,72603-14503:Jonas Valanciunas
